<a href="https://colab.research.google.com/github/Muzhi1920/awesome-models/blob/main/00-basic_tricks/00_auc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

In [ ]:
label = [1,0,1,0,1,0]
pre = [0.9, 0.8, 0.4, 0.9, 0.66, 0.7]
print('sklearn:', roc_auc_score(label, pre))

sklearn: 0.2777777777777778


# 1. 分类任务AUC

In [ ]:
# 西瓜书推导，仅能用于分类任务0-1
def c_pnr(label, pre):
    pos = [i for i in range(len(label)) if label[i] > 0.5]
    neg = [i for i in range(len(label)) if label[i] < 0.5]
    auc = 0
    for i in pos:
        for j in neg:
            if pre[i] > pre[j]:
                auc += 1
            elif pre[i] == pre[j]:
                auc += 0.5
    return auc / (len(pos)*len(neg))

'auc:', c_pnr(label, pre)

('auc:', 0.2777777777777778)

## 1.回归任务PNR

- `label = [0.1,0.2,0.3,0.4]`
- `score = [0.6,0.5,0.7,0.8]`
- 样本构成的pari对`[(0.1,0.6),(0.2,0.5),(0.3,0.7),(0.4,0.8)]`
- pair对之间的比较：一共`N(N-1)`；
  - 正序：`label_A > label_B`时，`score_A > score_B`；
  - 逆序：非同序。
- 统计正逆序对比：`(正序/逆序)`；或者`AUC：正序/(正序+逆序)，即正序数/(N*(N-1))`；




In [ ]:
def r_pnr(label, score):
  pos = 0
  N = len(label)
  for i, (yi, yi_) in enumerate(zip(label, score)):
    for j, (yj, yj_) in enumerate(zip(label, score)):
      if j <= i: # 构建所有可能的pair对 C_N^2
        continue
      if((yi - yj) * (yi_ - yj_) > 0 or (yi == yj and yi_ == yj_)): # 正序
        pos += 1
  return 2 * pos / (N * (N - 1))

arr1 = [0.1,0.2,0.3,0.4]
arr2 = [0.6,0.5,0.7,0.8]
'pnr :', r_pnr(arr1, arr2)

('pnr :', 0.8333333333333334)

## 结论
1. auc计算：tf只计算分类任务（使用积分方式，batch更新混淆矩阵）；
2. pnr计算：
  - 分类任务（正负分开，计算正逆序对比）
  - 回归任务（$C_N^2$种pari对比较）

如有问题，欢迎指出